# Korrelation zwischen Tweets und Aktienkursen, Appendix

*Version 1.3, 02. Januar 2019*

Im Rahmen des Moduls Data Science mit Python (dsp) an der Fachhochschule Nordwestschweiz soll die Korrelation zwischen Tweets- und Aktienkursen aufgezeigt werden. Im folgenden Notebook sind weiterführende Ausführungen zur Datenaufbereitung und zusätzliche beschreibende Plots zur Unterstützung des Verständnisses aufgeführt.

### Table of Contents
Data collection  
&nbsp;&nbsp;&nbsp;Appendix A1: [Data processing](#sec_processing)  

Qualitative analysis  
&nbsp;&nbsp;&nbsp;Appendix B1: [Datatypes](#sec_datatypes)  
&nbsp;&nbsp;&nbsp;Appendix B2: [Categorical variable "source"](#sec_source)

Quantitative analysis  
&nbsp;&nbsp;&nbsp;Appendix C1: [Position and dispersion measures](#sec_position)  
&nbsp;&nbsp;&nbsp;Appendix C2: [Polarity characteristics](#sec_polarity)  
&nbsp;&nbsp;&nbsp;Appendix C3: [Explorative correlation overview](#sec_correlation)  
&nbsp;&nbsp;&nbsp;Appendix C4: [Quantitative correlations](#sec_correlationmatrix)

### Data processing <a name="sec_processing"></a>

Grundlage für die Analysen bilden zwei separate Datensätze - diese werden in den nachfolgenden Abschnitten genauer erläutert. Die verwendeten Daten können jederzeit neu erstellt werden. Dazu kann das Skript `load_data.py` im *util*-Verzeichnis der Applikation ausgeführt werden. Die Applikation wird mittels *config.py* im selben Verzeichnis konfiguriert. 

Um die Daten neu zu erstellen, kann folgender Befehl ausgeführt werden:

```
$ python load_data.py
```

#### Twitter Daten

Twitter ist ein soziales Netwerk, wo Benutzer in kurzen Nachrichten (sog. "Tweets") miteinander kommunizieren. Twitter stellt eine eigene API zur Verfügung wo Daten konsumiert werden können. Der nachstehend verwendete Datensatz umfasst 6465 Tweets, welche entweder von den Benutzern *Tesla* oder *Elon Musk* geteilt oder mit einer @-Annotation an diese gerichtet wurden.

In [ ]:
!head -15 data/twitterdata.csv

Bevor mit den Daten gearbeitet wird, werden diese aufbereitet und in der gewünschten Form in DataFrames geladen. Die dafür notwendigen Funktionen sind im Modul DataHelper implementiert:

In [ ]:
from util.data_helper import DataHelper as dataHelper

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF8 -*-

# Author: Marco Romanutti -- marco.romanutti@students.fhnw.ch
# https://gitlab.fhnw.ch/ML-AT-FHNW/dsp_data_stories.git
# Code of corresponding helper classes, see:
# hs2018.twitter_stock_correlation.util

from IPython.display import display, HTML
import warnings; warnings.simplefilter('ignore') # Warninigs verified

# Load twitter data
raw_tweets = dataHelper.load_twitter_data(dataHelper)
# Set DatetimeIndex
indexed_tweets = dataHelper.set_datetimeindex(raw_tweets, 'Date of tweet')
# Clean tweets
cleaned_tweets = dataHelper.remove_specialcharacters(indexed_tweets)
# Lowercase tweets
lowercased_tweets = dataHelper.lowercase_text(cleaned_tweets, 'Tweets')
# Categorize source
categorized_tweets = dataHelper.create_category(lowercased_tweets, 'Source')
# Rename columns
renamed_tweets = dataHelper.rename_tweet_columns(categorized_tweets)
# Drop old data
new_tweets = dataHelper.drop_old_data(renamed_tweets)

# Display results
display(new_tweets)

Um Aussagen über den Inhalt zu machen, werden die einzelnen Tweet analysiert. Mithilfe der `TextBlob`-Library wird für jeden Tweet ein Polaritäts-Wert berechnet. Die Polarität beschreibt, wie positiv respektive negativ eine Nachricht einzuschätzen ist (vgl. https://en.wikipedia.org/wiki/Sentiment_analysis). Dazu wird das DataFrame um eine Variable ergänzt, welche Werte im Bereich von $-1.0 \dots, 1.0$ annehmen kann. Der Wert $-1.0$ steht für eine negative Haltung gegenüber dem Empfänger der Nachricht, der Wert $1.0$ für eine positive Haltung.

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF8 -*-

# Author: Marco Romanutti -- marco.romanutti@students.fhnw.ch
# https://gitlab.fhnw.ch/ML-AT-FHNW/dsp_data_stories.git
# Code of corresponding helper classes, see:
# hs2018.twitter_stock_correlation.util

# Analyze tweets
analyzed_tweets = dataHelper.get_sentiment_score(new_tweets)
# Display results
display(analyzed_tweets.polarity)

Wir erhalten ein DatetimeIndex-indexiertes DataFrame mit den folgenden Variabeln:

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF8 -*-

# Author: Marco Romanutti -- marco.romanutti@students.fhnw.ch
# https://gitlab.fhnw.ch/ML-AT-FHNW/dsp_data_stories.git
# Code of corresponding helper classes, see:
# hs2018.twitter_stock_correlation.util

# Copy DataFrame for further usage
tweets = analyzed_tweets
# Display results
list(tweets)

#### NASDAQ-100 Daten

Der NASDAQ-100 ist ein Aktienindex. Er besteht aus den Aktien der 100 Nicht-Finanzunternehmen mit der höchsten Marktkapitalisierung in den Vereinigten Staaten. Yahoo stellt eine API zum Herunterladen von Kursdaten zur Verfügung. Der nachfolgende Datensatz umfasst die Kursinformationen des Unternehmens Tesla aus dem Jahr 2018.

In [ ]:
!head -16 data/yahoodata.csv

Bevor mit den Daten gearbeitet wird, werden diese aufbereitet und in der gewünschten Form in Dataframes geladen. Die notwendigen Funktionen dafür finden sich wiederum im Modul DataHelper.

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF8 -*-

# Author: Marco Romanutti -- marco.romanutti@students.fhnw.ch
# https://gitlab.fhnw.ch/ML-AT-FHNW/dsp_data_stories.git
# Code of corresponding helper classes, see:
# hs2018.twitter_stock_correlation.util

# load yahoo data
raw_stocks = dataHelper.load_stock_data(dataHelper)
# Set DatetimeIndex
indexed_stocks = dataHelper.set_datetimeindex(raw_stocks, 'Date')
# Calculate biggest variation of stock price
extended_stocks = dataHelper.get_daily_difference(indexed_stocks)
# Drop unused columns, as we want to focus on important ones
reduced_stocks = dataHelper.drop_column(extended_stocks, 'Adjusted closing price (USD)')
# Round to two decimal places
rounded_stocks = dataHelper.round_data(reduced_stocks)
# Rename columns
renamed_stocks = dataHelper.rename_stock_columns(rounded_stocks)
# Drop old data
new_stocks = dataHelper.drop_old_data(renamed_stocks)
        
# Display results
display(new_stocks)

Wir erhalten ein DatetimeIndex-indexiertes DataFrame mit den folgenden Variabeln:

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF8 -*-

# Author: Marco Romanutti -- marco.romanutti@students.fhnw.ch
# https://gitlab.fhnw.ch/ML-AT-FHNW/dsp_data_stories.git
# Code of corresponding helper classes, see:
# hs2018.twitter_stock_correlation.util

# Copy DataFrame for further usage
stocks = new_stocks
# Display results
list(stocks)

Um Analysen über beide Datensets machen zu können, wird ein drittes, kombiniertes DataFrame erstellt. Der Twitter-Datensatz liegt in einer Granularität von Sekunden vor, während die NASDAQ100-Daten nur auf täglicher Basis vorhanden sind. Um das kombinierte DataFrame zu erstellen wird deshalb der Twitter-Datensatz resampled. Dies wird mit der Funktion `resample_tweets` gemacht. Im unstenstehenden Codeausschnit ist ersichtlich, mithilfe welcher Aggregatsfunktionen die einzelnen Variabeln zusammengezählt wurden.

```python
    def resample_tweets(data):
        """
        Function to resample all tweets by day.

        :param data: Original tweets with a granularity of seconds
        :return: Grouped tweets with indexed and grouped by a granularity of a day
        """
        # Resample Data
        resampled = pd.DataFrame()
        resampled['number_of_tweets'] = data.tweet.resample('D').count()
        resampled['length'] = data.length.resample('D').mean()
        resampled['likes'] = data.likes.resample('D').sum()
        resampled['retweets'] = data.retweets.resample('D').sum()
        resampled['polarity'] = data.polarity.resample('D').mean()
        
        return resampled
```

Die resampelten Daten können schliesslich kombiniert werden.

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF8 -*-

# Author: Marco Romanutti -- marco.romanutti@students.fhnw.ch
# https://gitlab.fhnw.ch/ML-AT-FHNW/dsp_data_stories.git
# Code of corresponding helper classes, see:
# hs2018.twitter_stock_correlation.util

# Prepare combined data
resampled_tweets = dataHelper.resample_tweets(tweets)
# Join data
combined_data = dataHelper.concatenate_data(resampled_tweets, stocks)

# Display results
display(combined_data)

Wir erhalten ein drittes DataFrame mit folgenden Variabeln:

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF8 -*-

# Author: Marco Romanutti -- marco.romanutti@students.fhnw.ch
# https://gitlab.fhnw.ch/ML-AT-FHNW/dsp_data_stories.git
# Code of corresponding helper classes, see:
# hs2018.twitter_stock_correlation.util

# Copy DataFrame for further usage
data = combined_data
# Display results
list(data)

### Dataypes <a name="sec_datatypes"></a>

Im folgenden Abschnitt wird nochmals auf die qualitativen Eigenschaften der drei DataFrames aus der Erhebung eingegangen. Die notwendigen Funktionen dafür finden sich im Modul StatHelper. Die Funktion `describe_types` liefert die Datentypen der Variabeln.

In [ ]:
from util.stat_helper import StatHelper as statHelper

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF8 -*-

# Author: Marco Romanutti -- marco.romanutti@students.fhnw.ch
# https://gitlab.fhnw.ch/ML-AT-FHNW/dsp_data_stories.git
# Code of corresponding helper classes, see:
# hs2018.twitter_stock_correlation.util

# print qualitative infos
print("Twitter dataset:")
display(statHelper.describe_types(tweets))

print("NASDAQ-100 dataset:")
display(statHelper.describe_types(stocks))

print("Combined dataset:")
display(statHelper.describe_types(data))

### Categorical variable "source" <a name="sec_source"></a>

Eine weitere Erläuterung wird bei der Variable Source benötigt. Darin wird ausgewiesen, von welchem Kanal die Twitternachricht abgesetzt wurde. Der untenstehende Plot beschreibt, welche Kanäle im Datenset vorkommen und wie oft dass die einzelnen Kanäle für die Tweets verwendet wurden.

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF8 -*-

# Author: Marco Romanutti -- marco.romanutti@students.fhnw.ch
# https://gitlab.fhnw.ch/ML-AT-FHNW/dsp_data_stories.git
# Code of corresponding helper classes, see:
# hs2018.twitter_stock_correlation.util

#import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt

# set plot styles
%matplotlib inline
plt.style.use('seaborn-muted')

# plot number of tweets per category
statHelper.plot_source_frequency(tweets)

### Position and dispersion measures <a name="sec_position"></a>

Im folgenden Abschnitt wird eine Übersicht der wichtigsten Lage- und Streuungsparameter gemacht. Der Fokus liegt dabei auf jeden Variabeln, mit welchen die Fragestellung inwiefern Tweets Rückschlüsse auf Aktienkurse zulassen beantwortet werden soll.

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF8 -*-

# Author: Marco Romanutti -- marco.romanutti@students.fhnw.ch
# https://gitlab.fhnw.ch/ML-AT-FHNW/dsp_data_stories.git
# Code of corresponding helper classes, see:
# hs2018.twitter_stock_correlation.util

# measures of location and distribution
print("Twitter dataset:")
display(statHelper.describe_data(tweets))

print("NASDAQ-100 dataset:")
display(statHelper.describe_data(stocks))

### Polarity characteristics <a name="sec_polarity"></a>

Um die Polarität der Tweets besser einordnen zu können, wird diese Variable etwas genauer untersucht. Bei Betrachtung des Historgrams, resp. der kumulativen Verteilung fällt auf, das die Tweets tendenziell positiv ausfallen (0 = neutral): 

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF8 -*-

# Author: Marco Romanutti -- marco.romanutti@students.fhnw.ch
# https://gitlab.fhnw.ch/ML-AT-FHNW/dsp_data_stories.git
# Code of corresponding helper classes, see:
# hs2018.twitter_stock_correlation.util

# Plot histogram, normalized histogram 
# and cumulated distribution of polarity
statHelper.plot_polarity_histogram(tweets)

Die Entwicklung der Polarität entlang einer Zeitachse lässt den Schluss zu, dass sich die Polarität über die Zeit gesehen einigermassen konstant verhält. Im Diagramm sind positive Polaritätswerte blau, negative hingegen rot eingefärbt.

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF8 -*-

# Author: Marco Romanutti -- marco.romanutti@students.fhnw.ch
# https://gitlab.fhnw.ch/ML-AT-FHNW/dsp_data_stories.git
# Code of corresponding helper classes, see:
# hs2018.twitter_stock_correlation.util

import altair as alt
alt.renderers.enable('default')

# Plot development of polarity
statHelper.plot_polarity_development(combined_data)

### Explorative correlation overview <a name="sec_correlation"></a>

Um die eigentliche Fragestellung zu beantworten, werden Zusammenhänge zwischen den beiden Datensets gesucht. Die `seaborn`-Library beinhaltet die notwendige Funktionalität, um solche Zusammenhänge explorativ zu entdecken. Aus Gründen der Übersichtlichkeit wird auf ein Titel bei den einzelnen Plots verzichtet.

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF8 -*-

# Author: Marco Romanutti -- marco.romanutti@students.fhnw.ch
# https://gitlab.fhnw.ch/ML-AT-FHNW/dsp_data_stories.git
# Code of corresponding helper classes, see:
# hs2018.twitter_stock_correlation.util

# Plot correlations
statHelper.plot_explorative_correlations(combined_data)

### Quantitative correlations <a name="sec_correlationmatrix"></a>

Die Korrelations-Plots aus dem vorigen Kapitel lassen bereits die Vermutung zu, dass die Twitter- und Aktiendaten nicht besonders korrelieren. Mithilfe der folgenden Korrelationsmatrix wird die Vermutung bestätigt. Im Plot ist jeweils der errechnete Koerrelationskoeffizient zweier Variabeln farblich dargestellt.

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF8 -*-

# Author: Marco Romanutti -- marco.romanutti@students.fhnw.ch
# https://gitlab.fhnw.ch/ML-AT-FHNW/dsp_data_stories.git
# Code of corresponding helper classes, see:
# hs2018.twitter_stock_correlation.util

# Plot correlation matrix
statHelper.plot_correlation_matrix(combined_data)